## 1. Import Libraries

In [1]:
!pip install xgboost

In [2]:
!pip install feature-engine

In [5]:
import os

import pickle

import warnings

import numpy as np

import pandas as pd

import xgboost as xgb

import sklearn
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
	OneHotEncoder,
	OrdinalEncoder,
	StandardScaler,
	MinMaxScaler,
	PowerTransformer,
	FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

# import sagemaker
# from sagemaker.estimator import Estimator
# from sagemaker.inputs import TrainingInput
# from sagemaker.tuner import (
#     IntegerParameter,
#     ContinuousParameter,
#     HyperparameterTuner
# )

## 2. Display Settings

In [7]:
pd.set_option("display.max_columns", None)

In [8]:
sklearn.set_config(transform_output="pandas")

In [9]:
warnings.filterwarnings("ignore")

## 3. Read Datasets

In [16]:
train = pd.read_csv(r"D:\Projects\Machine Learning\Flight Price Prediction\data\train.csv")
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-09,Kolkata,Banglore,14:05:00,12:00:00,1315,1.0,In-flight meal not included,9663
1,Jet Airways,2019-06-06,Kolkata,Banglore,18:55:00,04:40:00,585,1.0,In-flight meal not included,7757
2,Indigo,2019-05-12,Banglore,Delhi,10:10:00,13:00:00,170,0.0,No Info,3943
3,Air Asia,2019-04-09,Banglore,Delhi,04:55:00,07:45:00,170,0.0,No Info,4282
4,Spicejet,2019-06-01,Kolkata,Banglore,06:55:00,09:30:00,155,0.0,No Info,3873
...,...,...,...,...,...,...,...,...,...,...
635,Spicejet,2019-04-15,Delhi,Cochin,08:45:00,13:15:00,270,1.0,No Info,5830
636,Multiple Carriers,2019-06-01,Delhi,Cochin,12:50:00,01:30:00,760,1.0,No Info,6795
637,Jet Airways,2019-05-12,Kolkata,Banglore,20:25:00,18:00:00,1295,1.0,No Info,14151
638,Indigo,2019-06-12,Chennai,Kolkata,22:05:00,00:25:00,140,0.0,No Info,5277


In [17]:
val = pd.read_csv(r"D:\Projects\Machine Learning\Flight Price Prediction\data\val.csv")
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-27,Delhi,Cochin,16:00:00,04:25:00,745,1.0,In-flight meal not included,12898
1,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
2,Air Asia,2019-05-24,Kolkata,Banglore,07:35:00,23:30:00,955,1.0,No Info,5192
3,Spicejet,2019-05-09,Banglore,Delhi,21:10:00,00:05:00,175,0.0,No check-in baggage included,4319
4,Indigo,2019-05-24,Banglore,Delhi,04:00:00,06:50:00,170,0.0,No Info,3943
...,...,...,...,...,...,...,...,...,...,...
155,Air India,2019-05-24,Kolkata,Banglore,05:50:00,20:25:00,875,2.0,No Info,11201
156,Vistara,2019-05-21,Kolkata,Banglore,20:20:00,22:40:00,1580,1.0,No Info,8452
157,Spicejet,2019-05-06,Mumbai,Hyderabad,22:45:00,00:15:00,90,0.0,No check-in baggage included,2175
158,Indigo,2019-06-03,Banglore,Delhi,00:25:00,03:15:00,170,0.0,No Info,3943


In [18]:
test = pd.read_csv(r"D:\Projects\Machine Learning\Flight Price Prediction\data\test.csv")
test

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-04-24,Delhi,Cochin,08:45:00,13:15:00,270,1.0,No check-in baggage included,4760
1,Air India,2019-05-01,Kolkata,Banglore,12:00:00,11:10:00,1390,2.0,No Info,11148
2,Multiple Carriers,2019-06-24,Delhi,Cochin,14:00:00,01:30:00,690,1.0,No Info,13377
3,Air India,2019-03-18,Mumbai,Hyderabad,06:20:00,07:40:00,80,0.0,No Info,3100
4,Air India,2019-06-24,Delhi,Cochin,12:00:00,19:15:00,435,2.0,No Info,9548
...,...,...,...,...,...,...,...,...,...,...
195,Jet Airways,2019-03-03,Mumbai,Hyderabad,02:55:00,04:25:00,90,0.0,No Info,5678
196,Spicejet,2019-04-15,Banglore,Delhi,21:10:00,00:05:00,175,0.0,No check-in baggage included,3919
197,Jet Airways,2019-06-09,Delhi,Cochin,16:00:00,04:25:00,745,1.0,In-flight meal not included,10577
198,Indigo,2019-06-01,Banglore,Delhi,16:55:00,19:45:00,170,0.0,No Info,4823


## 4. Preprocessing Operations

In [19]:
# airline
air_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
    ("scaler", MinMaxScaler())
])

# source & destination
location_pipe1 = Pipeline(steps=[
    ("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
    ("encoder", MeanEncoder()),
    ("scaler", PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
    return (
        X
        .assign(**{
            f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
            for col in columns
        })
        .drop(columns=columns)
    )

location_transformer = FeatureUnion(transformer_list=[
    ("part1", location_pipe1),
    ("part2", FunctionTransformer(func=is_north))
])

# dep_time & arrival_time
time_pipe1 = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
    ("scaler", MinMaxScaler())
])

def part_of_day(X, morning=4, noon=12, eve=16, night=20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col: pd.to_datetime(X.loc[:, col]).dt.hour
        for col in columns
    })

    return (
        X_temp
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_temp.loc[:, col].between(morning, noon, inclusive="left"),
                 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
                 X_temp.loc[:, col].between(eve, night, inclusive="left")],
                ["morning", "afternoon", "evening"],
                default="night"
            )
            for col in columns
        })
        .drop(columns=columns)
    )

time_pipe2 = Pipeline(steps=[
    ("part", FunctionTransformer(func=part_of_day)),
    ("encoder", CountFrequencyEncoder()),
    ("scaler", MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list=[
    ("part1", time_pipe1),
    ("part2", time_pipe2)
])

# duration
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma


    def fit(self, X, y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include="number").columns.to_list()

        self.reference_values_ = {
            col: (
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values
                .reshape(-1, 1)
            )
            for col in self.variables
        }

        return self


    def transform(self, X):
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
            obj = pd.DataFrame(
                data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
                columns=columns
            )
            objects.append(obj)
        return pd.concat(objects, axis=1)
    

def duration_category(X, short=180, med=400):
    return (
        X
        .assign(duration_cat=np.select([X.duration.lt(short),
                                    X.duration.between(short, med, inclusive="left")],
                                    ["short", "medium"],
                                    default="long"))
        .drop(columns="duration")
    )

def is_over(X, value=1000):
    return (
        X
        .assign(**{
            f"duration_over_{value}": X.duration.ge(value).astype(int)
        })
        .drop(columns="duration")
    )

duration_pipe1 = Pipeline(steps=[
    ("rbf", RBFPercentileSimilarity()),
    ("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
    ("cat", FunctionTransformer(func=duration_category)),
    ("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
    ("part1", duration_pipe1),
    ("part2", duration_pipe2),
    ("part3", FunctionTransformer(func=is_over)),
    ("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
    ("imputer", SimpleImputer(strategy="median")),
    ("union", duration_union)
])

# total_stops
def is_direct(X):
    return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("", FunctionTransformer(func=is_direct))
])

# additional_info
info_pipe1 = Pipeline(steps=[
    ("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
    return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
("part1", info_pipe1),
("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
("union", info_union)
])

# column transformer
column_transformer = ColumnTransformer(transformers=[
("air", air_transformer, ["airline"]),
("doj", doj_transformer, ["date_of_journey"]),
("location", location_transformer, ["source", 'destination']),
("time", time_transformer, ["dep_time", "arrival_time"]),
("dur", duration_transformer, ["duration"]),
("stops", total_stops_transformer, ["total_stops"]),
("info", info_transformer, ["additional_info"])
], remainder="passthrough")

# feature selector
estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
estimator=estimator,
scoring="r2",
threshold=0.1
) 

# preprocessor
preprocessor = Pipeline(steps=[
("ct", column_transformer),
("selector", selector)
])

In [20]:
preprocessor.fit(
    train.drop(columns="price"),
    train.price.copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Other',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetime...
                                                                                                                                     replace_with='Other',
                                                                                                                                     tol=0.1)),
                                                                                                                   ('encoder',
                                                                                                                    OneHotEncoder(handle_unknown='ignore',
                                                                                                                                  sparse_output=False))])),
                                                                                                  ('part2',
                                                                                                   FunctionTransformer(func=<function have_info at 0x000001E54BE6DF80>))]))]),
                                                  ['additional_info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=42),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [21]:
preprocessor.transform(train.drop(columns="price"))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_Other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,dur__duration_rbf_25,dur__duration_cat,dur__duration,stops__total_stops,stops__is_direct_flight
0,0.0,1.0,0.0,0.588235,0.584746,-0.421157,-0.434646,-0.385350,2.0,1.398893,1.0,0
1,0.0,1.0,0.0,0.823529,0.822034,-0.421157,-0.434646,-0.385350,2.0,-0.089327,1.0,0
2,1.0,0.0,0.0,0.588235,0.610169,-0.756979,-1.579379,2.478883,0.0,-0.935369,0.0,1
3,0.0,0.0,1.0,0.352941,0.330508,-0.756979,-1.579379,2.478883,0.0,-0.935369,0.0,1
4,0.0,0.0,1.0,0.764706,0.779661,-0.421157,-0.434646,-0.385350,0.0,-0.965949,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
635,0.0,0.0,1.0,0.411765,0.381356,1.089421,0.741943,-0.385350,1.0,-0.731503,1.0,0
636,0.0,0.0,0.0,0.764706,0.779661,1.089421,0.741943,-0.385350,2.0,0.267438,1.0,0
637,0.0,1.0,0.0,0.588235,0.610169,-0.421157,-0.434646,-0.385350,2.0,1.358119,1.0,0
638,1.0,0.0,0.0,0.882353,0.872881,-1.919848,-1.583516,-0.385350,0.0,-0.996529,0.0,1


In [ ]:
tuner.fit(data_channels)

## 8. Model Evaluation

In [ ]:
with open("xgboost-model", "rb") as f:
    best_model = pickle.load(f)
    
best_model

In [ ]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [ ]:
evaluate_model("train")

In [ ]:
evaluate_model("val")

In [ ]:
evaluate_model("test")